<p align="center">
    <h1> NVIDIA Kaolin - CVPR 2025 Tutorial </h1>
</p>

<table style="background:white; border-collapse:collapse; width:auto;">
  <tr>
    <td colspan="2" align="center">
      <img src="../../../assets/nvidia-logo-horz.png" style="height:200px; vertical-align:middle;">
    </td>
  </tr>
  <tr>
    <td valign="middle" style="text-align:center; padding:10px; background:white !important; border:none !important;"><b>Rendering powered by:</b></td>
    <td style="text-align:center; padding:10px; background:white !important; border:none !important;">
    3dgrut, NVIDIA Optix, NVIDIA Slang
    </td>
  </tr>
  <tr>
    <td valign="middle"><b>Simulations powered by:</b></td>
    <td>
    NVIDIA Warp
    </td>
  </tr>
</table>

*Note: This tutorial was tested on Ubuntu 20.04. Full support for Windows OS is not guaranteed.*

*Note 2: It is recommended to run cells one-by-one, Jupyter may run out of sync when running all cells at once.*

-----

**In the following, you will learn how to use NVIDIA kaolin to jointly ray trace & simulate volumetric radiance fields and meshes.** <br>
**Throughout the tutorial, we will use various frameworks from the NVIDIA software stack, demonstrating how Kaolin can be used to integrate these useful tools together.**

**Relevant Literature:**
* [Simplicits: Mesh-Free, Geometry-Agnostic, Elastic Simulation [Modi et al. 2024]](https://research.nvidia.com/labs/toronto-ai/simplicits/)
* [3D Gaussian Ray Tracing: Fast Tracing of Particle Scenes [Moenne-Loccoz et al. 2024]](https://gaussiantracer.github.io/)
* [3DGUT: Enabling Distorted Cameras and Secondary Rays in Gaussian Splatting [Wu et al. 2025]](https://research.nvidia.com/labs/toronto-ai/3DGUT/)

## Requirements for this demo

1. [Follow 3dgrut installation instructions.]("https://github.com/nv-tlabs/3dgrut?tab=readme-ov-file#-1-dependencies-and-installation")
2. Install additional requirements:

In [ ]:
!pip install matplotlib ipywidgets k3d --quiet

In [ ]:
import copy
import logging
import numpy as np
import os
import sys
import time
import torch
import torchvision.transforms.functional as F
import kaolin
from matplotlib import pyplot as plt
from gaussian_utils import transform_gaussians_lbs, pad_transforms, PHYS_NOTEBOOKS_DIR

logging.basicConfig(level=logging.INFO, stream=sys.stdout, format="%(asctime)s|%(levelname)8s| %(message)s")
logger = logging.getLogger(__name__)

def log_tensor(t, name, **kwargs):
    """ Debugging util, e.g. call: log_tensor(t, 'my tensor', print_stats=True) """
    logger.info(kaolin.utils.testing.tensor_info(t, name=name, **kwargs))

%load_ext autoreload
%autoreload 2

# Renderer (3DGRUT)

## Gaussian Splat and Mesh Assets

Our demo starts by loading a pretrained 3dgut or 3dgrt Gaussian Splat models. We will grab these assets from AWS below, but you can also train your own. <br>
More details about training are available in the official github: https://github.com/nv-tlabs/3dgrut

*3dgut is now being presented in the CVPR 2025, and the code was recently released by NVIDIA.*  <br>

In [ ]:
# Data will be stored relative to this notebook location
root_data_path = os.path.join(PHYS_NOTEBOOKS_DIR, "local_data")
os.makedirs(root_data_path, exist_ok=True)
os.chdir(PHYS_NOTEBOOKS_DIR)

data_path = os.path.join(root_data_path, "grut_cvpr2025")

if os.path.exists(data_path):
    logger.info(f'Data already downloaded: {data_path}')
else:
    logger.info(f'Downloading and unzipping data')
    !wget https://nvidia-kaolin.s3.us-east-2.amazonaws.com/data/grut_cvpr2025_v3.zip -P local_data/; unzip local_data/grut_cvpr2025_v3.zip -d local_data/;

# Pre-trained Mixture-of-Gaussians objects to load.
# These objects can be trained with either 3dgut or 3dgrt, we will render them with 3dgrt's ray tracing here
# 1. A doll object, the focus of our demo
gs_object = os.path.join(data_path, "3dgrt", "BluehairRagdoll.ply")
# 2. A table object on a patch of grass, which will serve as our environment
gs_env = os.path.join(data_path, "3dgrt", "tools_revised2.ply")  # removed outliers

# Folder containing available mesh assets in obj, glb, gltf format
# Our demo app can load these meshes and move them around the scene
mesh_assets_folder = os.path.join(data_path, "mesh")
# Folder containing available envmaps in hdr format
# Where the gaussian env is incomplete, we will query this env-map as background
envmap_assets_folder = os.path.join(data_path, "envmaps")

# Default config to use for the gaussian object if the saved model doesn't include it
# (i.e. .ply files don't reference the original config)
default_config = "apps/colmap_3dgrt.yaml" 

## Setup 3dgrut Engine & Scene

3dgrut includes a hybrid rendering engine that can render Gaussian based radiance fields, and meshes with a joint path tracer. <br>
Let's set up the 3dgrut rendering engine, load the Gaussian objects and add a sample mesh for the fun of it.

Starting with the Gaussian objects, pretrained with 3dgrut or 3dgrt:

In [ ]:
from threedgrut_playground.engine import Engine3DGRUT
from threedgrut_playground.utils.composition import join_gaussians

# Start the engine with the env Gaussian object.
# The engine will set the scene scale and rescale added meshes according to the scene size
engine = Engine3DGRUT(
    gs_object=gs_env,
    mesh_assets_folder=mesh_assets_folder,
    envmap_assets_folder=envmap_assets_folder,
    default_config=default_config
)

# Load another 3dgrt Gaussian object, and translate up along z axis to position above table
# Then concatenate both Gaussians into a single 3dgrut object the engine can render
with torch.no_grad():
    env_mog = engine.scene_mog
    object_mog,_ = engine.load_3dgrt_object(gs_object, config_name=default_config)
    object_mog.positions[:,2] = object_mog.positions[:,2] + 1.3
    engine.scene_mog = join_gaussians(env_mog, object_mog)

Then we'll remove the default sphere primitive, and load another custom mesh primitive instead.

ℹ️ **Kaolin** is used internally here within `Engine3DGRUT`, to load the sample meshes with their materials.
Kaolin saves precious time of dealing with gltf format shenanigans, with a few lines of code the preprocessed mesh is loaded into tensors, compatible with the 3dgrut path tracer. <br> See here: https://github.com/nv-tlabs/3dgrut/blob/main/threedgrut_playground/utils/mesh_io.py#L132

In [ ]:
from threedgrut_playground.engine import OptixPrimitiveTypes

# The engine is loaded with a sample mesh primitive (glass sphere),
# remove the default mesh primitives from the engine here
for mesh_name in list(engine.primitives.objects.keys()):
    engine.primitives.remove_primitive(mesh_name)

# Instead, we'll add another mesh object to a scene.
# For now, we load and display it with a simple Lambertian shader.
# To respect the original look, we'll later display as a Physically Based Rendered (PBR) mesh.
print(f'Available meshes: {list(engine.primitives.assets.keys())}')
engine.primitives.add_primitive(
    geometry_type='Spray_bottle',
    primitive_type=OptixPrimitiveTypes.DIFFUSE, # Lambertian
    device='cuda'
)

# Take note of the mesh name, we'll need it to reference this primitive through the engine
mesh_name = list(engine.primitives.objects.keys())[0]
# Reference the mesh object, as OptixPrimitive, to quickly access the geometry and transforms later
prim = engine.primitives.objects[mesh_name]

# Position the mesh nicely within the scene, i.e. above the table
engine.primitives.objects[mesh_name].transform.tx += 0.95
engine.primitives.objects[mesh_name].transform.tz += 1.5
engine.primitives.objects[mesh_name].transform.rz += 25.0
engine.primitives.objects[mesh_name].transform.ry += 20.0

# Configure some initial rendering settings - to low and fast quality
engine.camera_type = 'Pinhole'
engine.camera_fov = 60.0
engine.use_spp = False                # Disable antialiasing for now
engine.antialiasing_mode = '4x MSAA'  # Set the default antialiasing to 4x MSAA, if enabled later
engine.use_optix_denoiser = False     # Disable Optix denoiser

# Set a HDR envmap as background, to provide some light to our mesh
engine.environment.set_env('drackenstein_quarry_puresky_4k.hdr')
engine.environment.ibl_intensity=0.60
engine.environment.exposure=-0.04
engine.environment.envmap_offset = [-0.5, -0.25]

# Finally, let engine know it should refresh internal structures due to changes
engine.invalidate_materials_on_gpu()  # Sync newly loaded materials (loaded with the mesh)
engine.rebuild_bvh(engine.scene_mog)                 # Rebuild gaussians BVH
engine.primitives.rebuild_bvh_if_needed(True, True)  # Rebuild meshes BVH

In [ ]:
# Uncomment to peek at documentation for further details about the 3dgrut rendering engine
# engine?

For the sake of the tutorial, we'll create some useful functions that quickly let us switch between fast, medium, and high quality settings.

* Fast mode is useful for interacting with this notebook.
* High quality mode is useful for, i.e. rendering and exporting a video.
* The medium quality mode strikes a balance between the two, and allows for a quick preview of the high quality settings.

In [ ]:
def set_fast_quality_mode():
    engine.use_spp = False
    engine.antialiasing_mode = '4x MSAA'
    engine.spp.mode = 'msaa'
    engine.spp.spp = 4
    engine.spp.reset_accumulation()
    engine.use_optix_denoiser = False

    # Use simple Lambertian shading for all mesh objects
    for prim in engine.primitives.objects.values():
         prim.primitive_type = OptixPrimitiveTypes(OptixPrimitiveTypes.DIFFUSE)

def set_medium_quality_mode():
    engine.use_spp = True
    engine.antialiasing_mode = '8x MSAA'
    engine.spp.mode = 'msaa'
    engine.spp.spp = 4
    engine.spp.reset_accumulation()
    engine.use_optix_denoiser = True

    # Use Path Tracing for all mesh objects
    for prim in engine.primitives.objects.values():
         prim.primitive_type = OptixPrimitiveTypes(OptixPrimitiveTypes.PBR)

def set_high_quality_mode():
    # Make sure engine renders high quality frames
    engine.use_spp = True
    engine.antialiasing_mode = 'Quasi-Random (Sobol)'
    engine.spp.mode = 'low_discrepancy_seq'
    engine.spp.spp = 64
    engine.spp.reset_accumulation()
    engine.use_optix_denoiser = True

    # Use Path Tracing for all mesh objects
    for prim in engine.primitives.objects.values():
         prim.primitive_type = OptixPrimitiveTypes(OptixPrimitiveTypes.PBR)

Test the engine we loaded -- render a single frame:

In [ ]:
from PIL import Image

# Render a frame from the engine
camera = kaolin.render.easy_render.default_camera(512)
renderbuffer = engine.render(camera)\

# Convert to PIL so Jupyter can display it
rgb_buffer = renderbuffer['rgb']
rgb_buffer = (rgb_buffer[0] * 255).to(torch.uint8)
image = Image.fromarray(rgb_buffer.cpu().numpy())

display(image)

## Interactive Renderer

Rendering is more fun when it becomes interactive! Hence, next we'll add some camera control..

ℹ️ Here we use **Kaolin**'s "easy render" visualizer to drive the 3dgrut engine. <br>
The visualizer is a quick way for rendering frames and interacting with the camera within the Jupyter notebook.

Internally, the 3dgrut engine uses NVIDIA Slang & Optix to quickly ray trace gaussians and meshes within the scene.

In [ ]:
# Start by setting up a kaolin camera object --

def reset_camera(_camera):
    # Position the Kaolin camera to observe the table
    _camera.update(torch.tensor(
        [[-0.51,   0.86,  0.00,  0.00],
         [-0.25,  -0.14,  0.96,  0.02],
         [ 0.83,   0.49,  0.29, -7.31],
         [ 0.00,   0.00,  0.00,  1.00]],
        dtype=_camera.dtype, device=_camera.device
    ))
    
# Create initial camera
camera = kaolin.render.easy_render.default_camera(512)
# Set it to Blender coordinates (Z axis pointing upwards)
camera.change_coordinate_system(
    torch.tensor([[1, 0, 0],
                  [0, 0, 1],
                  [0, -1, 0]]
))
camera = camera.cuda()
# ..and position above objects of interest
reset_camera(camera)

Kaolin's Visualizer requires two rendering functions:
* `fast_render()` - called during user interactions. Should be able to return frames quickly.
* `render()` - called when the user stops interacting with the scene to generate higher quality frames.

These functions integrate well with 3dgrut's engine:
We'll set `fast_render()` to render a single-pass frame, without fancy effects. <br>
Then `render()` will issue a multi-pass frame, where more complex effects like multisampling antialiasing and depth-of-field can be rendered.

In [ ]:
def fast_render(in_cam, **kwargs):
    # Called during interactions, disables effects for quick rendering
    framebuffer = engine.render_pass(in_cam, is_first_pass=True)
    # Alpha channel is the opacity output map from the renderer
    if engine.environment.is_ignore_envmap():
        alpha = framebuffer['opacity']
    else: # However if using an env map - alpha channel is always 1, since we render the background
        alpha = torch.ones_like(framebuffer['opacity'])
    # Read back the rendered rgb buffer and convert to a format supported by the visualizer
    rgba_buffer = torch.cat([framebuffer['rgb'], alpha], dim=-1)
    rgba_buffer = torch.clamp(rgba_buffer, 0.0, 1.0)
    return (rgba_buffer[0] * 255).to(torch.uint8) # Convert to RGBA image of uint8 pixels of [0,255]


def render(in_cam, **kwargs):
    # Called when the user stops interacting, to generate a high quality frame
    is_first_pass = engine.is_dirty(in_cam)
    framebuffer = engine.render_pass(in_cam, is_first_pass=True)
    # Note: this loop will stall until all passes are done
    while engine.has_progressive_effects_to_render():
        framebuffer = engine.render_pass(in_cam, is_first_pass=False)
    # Alpha channel is the opacity output map from the renderer
    if engine.environment.is_ignore_envmap():
        alpha = framebuffer['opacity']
    else: # However if using an env map - alpha channel is always 1, since we render the background
        alpha = torch.ones_like(framebuffer['opacity'])
    # Read back the rendered rgb buffer and convert to a format supported by the visualizer
    rgba_buffer = torch.cat([framebuffer['rgb'], alpha], dim=-1)
    rgba_buffer = torch.clamp(rgba_buffer, 0.0, 1.0)
    return (rgba_buffer[0] * 255).to(torch.uint8) # Convert to RGBA image of uint8 pixels of [0,255]


# Initialize the visualizer with the 2 render functions above, and a camera object.
visualizer = kaolin.visualize.IpyTurntableVisualizer(
    height=camera.height,
    width=camera.width,
    camera=camera,
    render=render,
    fast_render=fast_render,
    max_fps=8,
    world_up_axis=2
)
# without a GUI, showing the interactive renderer is as simple as:
# visualizer.show()

# But we want to include some widgets.. and therefore we'll include some extra code below

## Setup GUI Widgets

The next block is a technical part that adds ipywidgets, so users can control the visualizer from notebook -- feel free to skip the details.

---------------

#### Renderer Settings Widgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display

separator = widgets.HTML(
    value='<hr style="border: 1px dashed #ccc; margin: 10px 0;">',
    layout=widgets.Layout(width='100%')
)

In [ ]:
lq_button = widgets.Button(description="🚗💨 Fast Settings")
mq_button = widgets.Button(description="⚖️ Balanced Settings")
hq_button = widgets.Button(description="💎 High Quality Settings")

aa_checkbox = widgets.Checkbox(
    value=engine.use_spp,
    description='✨ Use Antialiasing'
)

aa_mode_combo = widgets.Dropdown(
    options=engine.ANTIALIASING_MODES,
    value='4x MSAA',
    description='Antialiasing Mode'
)

denoiser_checkbox = widgets.Checkbox(
    value=engine.use_optix_denoiser,
    description='🧠 Use Optix Denoiser'
)

spp_slider = widgets.IntSlider(
    value=engine.spp.spp,
    min=1,
    max=64,
    step=1,
    orientation='horizontal',
    description='Antialiasing SPP',
    disabled=(engine.spp.mode == 'msaa')
)

In [ ]:
envmap_combo = widgets.Dropdown(
    options=engine.environment.available_envmaps,
    value=engine.environment.current_name,
    description="🌅 Environment Map"
)

ibl_intensity_slider = widgets.FloatLogSlider(
    value=engine.environment.ibl_intensity,
    min=-3,
    max=3,
    step=0.01,
    orientation='horizontal',
    description='💡 IBL Intensity',
    disabled=(engine.environment.is_ignore_envmap())
)

exposure_slider = widgets.FloatSlider(
    value=engine.environment.exposure,
    min=-10.0,
    max=10.0,
    step=0.01,
    orientation='horizontal',
    description='📸 Exposure',
    disabled=(engine.environment.is_ignore_envmap())
)

env_offset_theta_slider = widgets.FloatSlider(
    value=engine.environment.envmap_offset[0],
    min=-0.5,
    max=+0.5,
    step=0.01,
    orientation='horizontal',
    description='Offset θ',
    continuous_update=True,
    readout=True,
    readout_format='.3f',
    disabled=(engine.environment.is_ignore_envmap())
)

env_offset_phi_slider = widgets.FloatSlider(
    value=engine.environment.envmap_offset[1],
    min=-0.5,
    max=+0.5,
    step=0.01,
    orientation='horizontal',
    description='Offset φ',
    continuous_update=True,
    readout=True,
    readout_format='.3f',
    disabled=(engine.environment.is_ignore_envmap())
)

In [ ]:
def on_render_setting_change(change):
    engine.use_spp = aa_checkbox.value

    engine.antialiasing_mode = aa_mode_combo.value
    if engine.antialiasing_mode == '4x MSAA':
        engine.spp.mode = 'msaa'
        engine.spp.spp = 4
    elif engine.antialiasing_mode == '8x MSAA':
        engine.spp.mode = 'msaa'
        engine.spp.spp = 8
    elif engine.antialiasing_mode == '16x MSAA':
        engine.spp.mode = 'msaa'
        engine.spp.spp = 16
    elif engine.antialiasing_mode == 'Quasi-Random (Sobol)':
        engine.spp.mode = 'low_discrepancy_seq'
        engine.spp.spp = spp_slider.value
    else:
        raise ValueError('unknown antialiasing mode')

    engine.spp.reset_accumulation()
    engine.use_optix_denoiser = denoiser_checkbox.value

    spp_slider.value = engine.spp.spp
    spp_slider.disabled = (engine.spp.mode == 'msaa')
    visualizer.render_update()

def on_envmap_change(change):
    engine.environment.tonemapper = 'None'
    engine.environment.set_env(envmap_combo.value)
    engine.environment.ibl_intensity = ibl_intensity_slider.value
    engine.environment.exposure = exposure_slider.value
    engine.environment.envmap_offset[0] = env_offset_theta_slider.value
    engine.environment.envmap_offset[1] = env_offset_phi_slider.value

    exposure_slider.disabled = engine.environment.is_ignore_envmap()
    env_offset_theta_slider.disabled = engine.environment.is_ignore_envmap()
    env_offset_phi_slider.disabled = engine.environment.is_ignore_envmap()
    visualizer.render_update()

aa_checkbox.observe(on_render_setting_change, names='value')
aa_mode_combo.observe(on_render_setting_change, names='value')
denoiser_checkbox.observe(on_render_setting_change, names='value')
spp_slider.observe(on_render_setting_change, names='value')

envmap_combo.observe(on_envmap_change, names='value')
ibl_intensity_slider.observe(on_envmap_change, names='value')
exposure_slider.observe(on_envmap_change, names='value')
env_offset_theta_slider.observe(on_envmap_change, names='value')
env_offset_phi_slider.observe(on_envmap_change, names='value')

quality_buttons = widgets.HBox([lq_button, mq_button, hq_button])
renderer_controls = widgets.VBox([quality_buttons, separator,
                                  denoiser_checkbox, aa_checkbox, aa_mode_combo, spp_slider, separator, 
                                  envmap_combo, ibl_intensity_slider, exposure_slider, env_offset_theta_slider, env_offset_phi_slider])

#### Object Transform Widgets

In [ ]:
_object_controls = []
shader_type_controls = []
prim_transform_controls = {}
transform_events_on = True

def reset_transform_widgets(prim_widgets, transform):
    prim_widgets['pos_x_slider'].value = transform.tx
    prim_widgets['pos_y_slider'].value = transform.ty
    prim_widgets['pos_z_slider'].value = transform.tz
    # Set rotation directly using the rx, ry, rz properties
    prim_widgets['rot_x_slider'].value = transform.rx
    prim_widgets['rot_y_slider'].value = transform.ry
    prim_widgets['rot_z_slider'].value = transform.rz 
    # Set scale directly using the sx, sy, sz properties
    prim_widgets['scale_slider'].value = transform.sx

def reset_transform_widgets_to_engine_values():
    global transform_events_on
    transform_events_on = False
    
    for prim_name, prim in engine.primitives.objects.items():
        transform = engine.primitives.objects[prim_name].transform

        if prim_name in prim_transform_controls:
            reset_transform_widgets(prim_transform_controls[prim_name], transform)
        else:
            logger.warn(f'Error? GUI controls for engine object {prim_name} not found')
    transform_events_on = True

In [ ]:
for prim_name, prim in engine.primitives.objects.items():

    prim_widgets = {
        'title': widgets.Label(value=f'🏺 {prim_name}')
    }

    def on_shader_change(change):
        if prim_widgets['type'].value == 'Lambertian':
            prim.primitive_type = OptixPrimitiveTypes(OptixPrimitiveTypes.DIFFUSE)
        elif prim_widgets['type'].value == 'Cook Torrance':
            prim.primitive_type = OptixPrimitiveTypes(OptixPrimitiveTypes.PBR)
        engine.primitives.recompute_stacked_buffers()
        engine.primitives.rebuild_bvh_if_needed(True, True)
        visualizer.render_update()
    
    prim_type_combo = widgets.Dropdown(
        options=['Lambertian', 'Cook Torrance'],
        value='Lambertian',
        description='Shader'
    )
    prim_type_combo.observe(on_shader_change, names='value')
    prim_widgets['type'] = prim_type_combo
    shader_type_controls.append([prim_type_combo, on_shader_change])
    
    def update_transform(change):
        # # Reset transform
        # engine.primitives.objects[prim_name].transform.reset()

        global transform_events_on
        if not transform_events_on:
            return
        
        # Set position directly using the tx, ty, tz properties
        engine.primitives.objects[prim_name].transform.tx = prim_widgets['pos_x_slider'].value
        engine.primitives.objects[prim_name].transform.ty = prim_widgets['pos_y_slider'].value
        engine.primitives.objects[prim_name].transform.tz = prim_widgets['pos_z_slider'].value
        # Set rotation directly using the rx, ry, rz properties
        engine.primitives.objects[prim_name].transform.rx = prim_widgets['rot_x_slider'].value
        engine.primitives.objects[prim_name].transform.ry = prim_widgets['rot_y_slider'].value
        engine.primitives.objects[prim_name].transform.rz = prim_widgets['rot_z_slider'].value
        # Set scale directly using the sx, sy, sz properties
        engine.primitives.objects[prim_name].transform.sx = prim_widgets['scale_slider'].value
        engine.primitives.objects[prim_name].transform.sy = prim_widgets['scale_slider'].value
        engine.primitives.objects[prim_name].transform.sz = prim_widgets['scale_slider'].value
        
        # Rebuild BVH if needed
        engine.primitives.rebuild_bvh_if_needed(force=True, rebuild=True)
        visualizer.render_update()
    
    for xyz in ['x', 'y', 'z']:
        prim_widgets[f'pos_{xyz}_slider'] = widgets.FloatSlider(
            value=getattr(prim.transform, f't{xyz}'),
            min=-5.0,
            max=5.0,
            step=0.1,
            description=f'Pos {xyz}:',
            continuous_update=True,
            readout=True,
            readout_format='.3f',
        )
    for xyz in ['x', 'y', 'z']:
        prim_widgets[f'rot_{xyz}_slider'] = widgets.FloatSlider(
            value=getattr(prim.transform, f'r{xyz}'),
            min=-180.0,
            max=180.0,
            step=0.1,
            description=f'Rotation {xyz}:',
            continuous_update=True,
            readout=True,
            readout_format='.3f',
        )

    prim_widgets[f'scale_slider'] = widgets.FloatSlider(
            value=prim.transform.sx,
            min=1e-3,
            max=8,
            step=0.001,
            description=f'Scale:',
            continuous_update=True,
            readout=True,
            readout_format='.3f',
    )
    for slider in prim_widgets.values():
        slider.observe(update_transform, names='value')
    prim_transform_controls[prim_name] = prim_widgets  # should be by reference
    _object_controls.append(widgets.VBox(tuple(prim_widgets.values())))

object_controls = widgets.VBox(_object_controls)

In [ ]:
def on_quality_change(quality):    
    aa_checkbox.unobserve(on_render_setting_change, names='value')
    aa_mode_combo.unobserve(on_render_setting_change, names='value')
    denoiser_checkbox.unobserve(on_render_setting_change, names='value')
    spp_slider.unobserve(on_render_setting_change, names='value')

    for control, event in shader_type_controls:
        control.unobserve(event, names='value')
        if quality in ('medium', 'high'):
            control.value = 'Cook Torrance'
        else:
            control.value = 'Lambertian'
    
    aa_checkbox.value = engine.use_spp
    denoiser_checkbox.value = engine.use_optix_denoiser 
    aa_mode_combo.value = engine.antialiasing_mode
    spp_slider.disabled = (engine.spp.mode == 'msaa')
    spp_slider.value = engine.spp.spp

    aa_checkbox.observe(on_render_setting_change, names='value')
    aa_mode_combo.observe(on_render_setting_change, names='value')
    denoiser_checkbox.observe(on_render_setting_change, names='value')
    spp_slider.observe(on_render_setting_change, names='value')

    for control, event in shader_type_controls:
        control.observe(event, names='value')
    
    engine.primitives.recompute_stacked_buffers()
    engine.primitives.rebuild_bvh_if_needed(True, True)
    visualizer.render_update()
def on_fast_quality_btn(b):
    set_fast_quality_mode()
    on_quality_change('fast')
def on_medium_quality_btn(b):
    set_medium_quality_mode()
    on_quality_change('medium')
def on_high_quality_btn(b):
    set_high_quality_mode()
    on_quality_change('high')
lq_button.on_click(on_fast_quality_btn)
mq_button.on_click(on_medium_quality_btn)
hq_button.on_click(on_high_quality_btn)

#### Materials Widget

In [ ]:
from matplotlib.colors import to_rgb, to_hex

def get_gui_rgb(color_picker):
    return to_rgb(color_picker.value)
def set_gui_rgb(color_picker, rgb):
    r, g, b = rgb[0].cpu().item(), rgb[1].cpu().item(), rgb[2].cpu().item()
    color_picker.value = to_hex([r, g, b])
    
available_pbr_materials = list(engine.primitives.registered_materials.keys())
selected_pbr_material = available_pbr_materials[-1]

pbr_materials_combo = widgets.Dropdown(
    options=list(engine.primitives.registered_materials.keys()),
    value=selected_pbr_material,
    description='🎨 PBR Material'
)

In [ ]:
diffuse_factor = engine.primitives.registered_materials[selected_pbr_material].diffuse_factor
diffuse_picker = widgets.ColorPicker(
    concise=True,
    description='Diffuse',
    value=to_hex([diffuse_factor[0].cpu().item(), diffuse_factor[1].cpu().item(), diffuse_factor[2].cpu().item()]),
    disabled=False
)

emissive_factor = engine.primitives.registered_materials[selected_pbr_material].emissive_factor
emissive_picker = widgets.ColorPicker(
    concise=True,
    description='Emissive',
    value=to_hex([emissive_factor[0].cpu().item(), emissive_factor[1].cpu().item(), emissive_factor[2].cpu().item()]),
    disabled=False
)

metallic_slider = widgets.FloatSlider(
    value=engine.primitives.registered_materials[selected_pbr_material].metallic_factor,
    min=0.0,
    max=1.0,
    step=0.01,
    orientation='horizontal',
    description='Metallicness',
    continuous_update=True,
    readout=True,
    readout_format='.3f'
)

roughness_slider = widgets.FloatSlider(
    value=engine.primitives.registered_materials[selected_pbr_material].roughness_factor,
    min=0.0,
    max=1.0,
    step=0.01,
    orientation='horizontal',
    description='Roughness',
    continuous_update=True,
    readout=True,
    readout_format='.3f'
)

ior_slider = widgets.FloatSlider(
    value=engine.primitives.registered_materials[selected_pbr_material].ior,
    min=0.0,
    max=1.0,
    step=0.01,
    orientation='horizontal',
    description='IOR',
    continuous_update=True,
    readout=True,
    readout_format='.3f'
)

In [ ]:
def on_selected_material_change(change):
    selected_pbr_material = pbr_materials_combo.value
    set_gui_rgb(diffuse_picker, engine.primitives.registered_materials[selected_pbr_material].diffuse_factor)
    set_gui_rgb(emissive_picker, engine.primitives.registered_materials[selected_pbr_material].emissive_factor)
    metallic_slider.value = engine.primitives.registered_materials[selected_pbr_material].metallic_factor
    roughness_slider.value = engine.primitives.registered_materials[selected_pbr_material].roughness_factor
    ior_slider.value = engine.primitives.registered_materials[selected_pbr_material].ior
    visualizer.render_update()

def on_material_factor_change(change):
    selected_pbr_material = pbr_materials_combo.value
    diffuse_factor = get_gui_rgb(diffuse_picker)
    emissive_factor = get_gui_rgb(emissive_picker)
    engine.primitives.registered_materials[selected_pbr_material].diffuse_factor[0] = diffuse_factor[0]
    engine.primitives.registered_materials[selected_pbr_material].diffuse_factor[1] = diffuse_factor[1]
    engine.primitives.registered_materials[selected_pbr_material].diffuse_factor[2] = diffuse_factor[2]
    engine.primitives.registered_materials[selected_pbr_material].emissive_factor[0] = emissive_factor[0]
    engine.primitives.registered_materials[selected_pbr_material].emissive_factor[1] = emissive_factor[1]
    engine.primitives.registered_materials[selected_pbr_material].emissive_factor[2] = emissive_factor[2]
    engine.primitives.registered_materials[selected_pbr_material].metallic_factor = metallic_slider.value
    engine.primitives.registered_materials[selected_pbr_material].roughness_factor = roughness_slider.value
    engine.primitives.registered_materials[selected_pbr_material].ior = ior_slider.value

    engine.invalidate_materials_on_gpu()
    visualizer.render_update()


pbr_materials_combo.observe(on_selected_material_change, names='value')

diffuse_picker.observe(on_material_factor_change, names='value')
emissive_picker.observe(on_material_factor_change, names='value')
metallic_slider.observe(on_material_factor_change, names='value')
roughness_slider.observe(on_material_factor_change, names='value')
ior_slider.observe(on_material_factor_change, names='value')

material_controls = widgets.VBox([pbr_materials_combo, diffuse_picker, emissive_picker, metallic_slider, roughness_slider, ior_slider])

#### Build the GUI component

In [ ]:
gui_tab = widgets.Tab()
tab_renderer = widgets.Box([renderer_controls])
tab_objects = widgets.Box([object_controls])
tab_materials = widgets.Box([material_controls])
gui_tab.children = [tab_renderer, tab_objects, tab_materials]
gui_tab.set_title(0, 'Render Settings')
gui_tab.set_title(1, 'Objects')
gui_tab.set_title(2, 'PBR Materials')
tab_renderer.layout = widgets.Layout(width='100%')
gui_tab.layout = widgets.Layout(width='50%')

---- ---

### Display the Visualizer + GUI

Feel free to experiment around with the object and engine settings through the GUI.

In [ ]:
visualizer.render_update() # Render initial frame
visualizer_box = widgets.HBox([visualizer.canvas, gui_tab])
display(visualizer_box)

Before continuing on to the next section, run the block below which prepares the mesh object.

In [ ]:
# Once happy with the mesh position, we bake the transform and rewrite the mesh coordinates.
# Later, simulation will take place in object coordinates
# Since we transform the mesh in world coordinates, we set the world coordinates as the new object coordinates.
baked_prim = prim.apply_transform()
baked_prim.transform.reset()
engine.primitives.objects[mesh_name] = baked_prim
prim = baked_prim

# GUI sliders in the UI below should reflect the new transform value
reset_transform_widgets_to_engine_values()

# Simulation (Simplicits)

ℹ️ **Kaolin** includes an entire module for simulating objects, using the [Simplicits](https://research.nvidia.com/labs/toronto-ai/simplicits/) method. <br>
The included implementation is further boosted with NVIDIA-warp.

# Prerequisites

Earlier we joined the doll and table gaussians into a single blob the engine can render.

However, simulating the combined gaussian scene requires segmentation, or rather, masking out which gaussians belong to the doll and table.

## Obtain Gaussian Segments

In [ ]:
# Combined gaussian object contains: [ Table Gaussian 1...  Table Gaussian N, Doll Gaussian 1, .. Doll Gaussian N]
table_mask = torch.zeros(len(env_mog) + len(object_mog), device=env_mog.device, dtype=torch.bool)
table_mask[:len(env_mog)] = True

doll_mask = torch.zeros(len(env_mog) + len(object_mog), device=env_mog.device, dtype=torch.bool)
doll_mask[len(env_mog):] = True

In [ ]:
# Shapes we'll simulate
doll_gaussians = engine.scene_mog[doll_mask]   # copy generated by indexing 3dgrut's MixtureOfGaussians model
table_gaussians = engine.scene_mog[table_mask] # copy generated by indexing 3dgrut's MixtureOfGaussians model
mesh = engine.primitives.objects[mesh_name]    # reference to OptixPrimitive

# To use these shapes with Simplicits, we need to sample points on them.
# As an approximation, we'll soon use the Gaussian means and mesh vertices.
print('')
print("Objects we'll use with Simplicits: \n ---------------------------------------- ")
print(f'Doll is made of {len(doll_gaussians)} Gaussians')
print(f'Table is made of {len(table_gaussians)} Gaussians')
print(f'Mesh is made of {len(mesh.vertices)} vertices')

## Material Parameters

We know the geometry of our objects (or at least, how to sample "cubature points" on them). <br>
Next we define some physical material properties for our objects, and some initial settings for Simplicits.

In [ ]:
mesh_obj_args ={ # Will be a rigid object
    "yms" : 1e5,     # Lower YM because its easier to converge and only 1 handle will exhibit numerical stiffening. 
    "prs" : 0.35,    # Metal has approximately 0.35 poisson ratio
    "rhos" : 500,    # Approximating density for the can of paint
    "appx_vol" : 0.5 # Approximating volume for paint can
}

doll_obj_args = { # Will be a trained, elastic object
    "yms" : 1e5,                # The doll can be trained at 1e5 youngs modulus. Later we can update this before simulation. 
    "prs" : 0.45,               # Rubber-like poisson ratio for doll
    "rhos" : 500,               # This density works well for elastic deformation
    "appx_vol" : 0.5,           # The volume is approximated, making it too low will affect convergence at fp32
    "num_handles" : 40,         # We want 40 control handles for a highly expressive object
    "model_layers" : 10,        # Number of simplicits MLP layers
    "training_num_steps" : 25000 # Training must be increased to 20k steps (or more) for 40 handles
}

## Sample Cubature Points & Densify Shape Volume

The Simplicits api in kaolin represents objects as cubature points.

Since both meshes and Gaussians are supported by points on the surface (or close to it, in the case of Volumetric Radiance Gaussian fields), we'll try to fill the volume of our shapes with additinal sampled points.

ℹ️ Fortunately, **Kaolin** includes a [densifier module](https://kaolin.readthedocs.io/en/latest/modules/kaolin.ops.gaussian.html#kaolin.ops.gaussian.sample_points_in_volume) that populates the gaussian shape with additional points.


#### Sample points from Mesh

In [ ]:
# Let's sample points on the mesh surface (volume would be better, but this works ok for more rigid shapes)
mesh_resampled_pts = kaolin.ops.mesh.sample_points(
            prim.vertices.unsqueeze(0), prim.triangles, prim.vertices.shape[0] * 3)[0].squeeze(0)
log_tensor(prim.vertices, 'mesh original pts', print_stats=True)
log_tensor(mesh_resampled_pts, 'mesh resampled pts', print_stats=True)

#### Sample points from 3D Gaussian field

In [ ]:
# Let's sample points inside the doll volume to make simulation more faithful
doll_resampled_pts = kaolin.ops.gaussian.sample_points_in_volume(
    xyz=doll_gaussians.get_positions().detach(), 
    scale=doll_gaussians.get_scale().detach(),
    rotation=doll_gaussians.get_rotation().detach(),
    opacity=doll_gaussians.get_density().detach(),
    clip_samples_to_input_bbox=False
)
log_tensor(doll_gaussians.get_positions(), 'doll orig pts', print_stats=True)
log_tensor(doll_resampled_pts, 'doll resampled pts', print_stats=True)

In [ ]:
# For the table, we will just use more opaque Gaussians, as sample_points_in_volume is defined for objects and not full environment fields
table_resampled_pts = table_gaussians.get_positions()[(table_gaussians.get_density() > 0.3).squeeze(), :]
log_tensor(table_gaussians.get_positions(), 'table and scene pts', print_stats=True)
log_tensor(table_resampled_pts, 'table and scene resampled pts', print_stats=True)

## Train Simplicits Skinning Weights / Initialize Rigid Objects



Let's start by creating some rigid SimplicitsObjects. Creating rigid objects is instantaneous.

In [ ]:
common_kwargs = {  # Some common starting parameters for materials (tune to preference)
    "yms": 1e6,    # This is a good starting point for a rubbery-object, in a reduced sim this is affected by numerical stiffness
    "prs": 0.45,   # Measures compressibility
    "rhos": 1000,  # Density of water, in a reduced sim, this is approximated due to numerical stiffness
    "appx_vol": 1  # Also close, but approximated. When this is too small and timestep is also small it can cause precision issues in our fp32 sims
}
other_kwargs = {}

# Let's initialize rigid objects
mesh_obj = kaolin.physics.simplicits.SimplicitsObject.create_rigid(
    mesh_resampled_pts, **mesh_obj_args, **other_kwargs)
logger.info(f'Created mesh rigid object')

# We may want to set a dynamically moving table to higher stiffness, but its a static object in this scene
table_obj = kaolin.physics.simplicits.SimplicitsObject.create_rigid(
    table_resampled_pts, **common_kwargs, **other_kwargs) 
logger.info(f'Created table rigid object')

Next we create an elastic SimplicitsObject and train its skinning weight functions using the volume samples from above. The simulator will then use these reduced degrees of freedom to drive the simulation.

In [ ]:
# One-liner to set up and train a Simplicits object..
# However, in the interest of time, you may run the block below to load a pretrained simplicits object.

# sim_obj = kaolin.physics.simplicits.SimplicitsObject.create_trained(
#     points,  # point samples
#     *args, **kwargs) 

**Note:** Since training of elastic objects takes a bit of time, we cache the result and reuse it next time we run the notebook:

In [ ]:
# Whether to save reduced degress of freedom used by the simulator and load from cache automatically
ENABLE_SIMPLICITS_CACHING = True # set to False to always retrain

cache_dir = os.path.join(PHYS_NOTEBOOKS_DIR, 'cache')
os.makedirs(cache_dir, exist_ok=True)
logger.info(f'Caching trained simplicits objects in {cache_dir}')

def train_or_load_simplicits_object(points, fname, *args, **kwargs):
    if not ENABLE_SIMPLICITS_CACHING or not os.path.exists(fname):
        logger.info('Training simplicits object. This will take 2-3min... ')
        start = time.time()

        # One-liner to set up Simplicits object
        sim_obj = kaolin.physics.simplicits.SimplicitsObject.create_trained(
            points,  # point samples
            *args, **kwargs) 
        
        end = time.time()
        logger.info(f"Ended training in {end-start} seconds")

        # We'll cache the result so we can quickly rerun the notebook.
        torch.save(sim_obj, fname)
        logger.info(f"Cached training result in {fname}")
    else:
        logger.info(f'Loading cached simplicits object from: {fname}')
        sim_obj = torch.load(fname, weights_only=False)
    return sim_obj

# Let's train doll as a deformable simplicits object
# For a high number like 40 handles, train with 20k steps at least
doll_obj = train_or_load_simplicits_object(
    doll_resampled_pts,
    os.path.join(cache_dir, 'simplicits_3dgrut_doll_20k.pt'),
    **doll_obj_args, **other_kwargs)

#### Visualize Sampled Cubature Points

Check that we sampled enough points on the object. <br>
Note how the purple gaussian object is solid due densification, and the red mesh object is hollow.

*Homework: try visualizing the trained simplicits weights of the doll as colored points (one weight at a time)!*

In [ ]:
import k3d
plot = k3d.plot()
plot += k3d.points(mesh_obj.pts.detach().cpu().numpy(), point_size=0.01, color=0xFF0000)
plot += k3d.points(doll_obj.pts.detach().cpu().numpy(), point_size=0.01, color=0x7700FF)
plot += k3d.points(table_obj.pts.detach().cpu().numpy(), point_size=0.01, color=0xCCCCCC)
plot.display()

## Setup Physics Scene

Here, we will reset initial conditions for the simulation and set up our forces using newest `kaolin.physics` API. 

In [ ]:
# The simulation material properties can be different from the ones used during training!
doll_obj.yms = 5e4*torch.ones_like(doll_obj.prs)
doll_obj.rhos = 500*torch.ones_like(doll_obj.prs)
mesh_obj.yms = 1e5*torch.ones_like(mesh_obj.prs)
mesh_obj.rhos = 200*torch.ones_like(mesh_obj.prs)

In [ ]:
# Create a default scene # default empty scene
scene = kaolin.physics.simplicits.SimplicitsScene()
# Convergence might not be guaranteed at few NM iterations, but runs very fast
scene.max_newton_steps = 4 
# Don't set too small when using small volumes (leads to fp precision issues)
scene.timestep=0.02 
# This helps the conditioning of the newton hessian, but approximating the hessian is worse for convergence
scene.newton_hessian_regularizer = 1e-5
# Use a direct dense solver in small scenes 
scene.direct_solve = True

# Add simulatable objects to the scene
mesh_obj_idx = scene.add_object(mesh_obj, num_qp=1000)
doll_obj_idx = scene.add_object(doll_obj, num_qp=2000) 
table_obj_idx = scene.add_object(table_obj, num_qp=15000, is_kinematic=True)

# Set gravity of the scene
scene.set_scene_gravity(acc_gravity=torch.tensor([0, 0, 9.8]))

# Setup collisions
collision_particle_radius = 0.05 # Currently a scene-wide radius at which collision energy starts to aggregate
scene.enable_collisions(
    collision_particle_radius=collision_particle_radius, 
    detection_ratio=1.25,                              # Detect collision within this multiple of the radius
    impenetrable_barrier_ratio=0.1,                    # Collision energy is infinite at this multiple of radius
    collision_penalty=500,                             # Coefficient for strength of collision response
    max_contact_pairs=20000,                           # Max number of contacting pairs allowed in scene
    friction=0.1                                       # Friction parameter for scene
)

## Simulate and Visualize

Now we are ready to simulate, but we still need to put in a bit of work to deform the Gaussians according to the simulation.

In [ ]:
# Save for resetting the simulation
mesh_orig_pos = prim.vertices.clone()
# Look up learned skinning weights using original doll positions
doll_skinning_weights = doll_obj.skinning_weight_function(doll_gaussians.get_positions())

def engine_refresh():
    """ Updates the 3dgrut engine with changes to objects """
    # Rebuild BVH of Gaussians
    engine.rebuild_bvh(engine.scene_mog)
    # Rebuild BVH of meshes
    engine.primitives.rebuild_bvh_if_needed(True, True)

def reset_simulation():
    # Reset simulator state back to initial conditions
    scene.reset_scene()
    # Set object to original coordinates
    engine.primitives.objects[mesh_name].transform.reset()
    scene.set_object_initial_transform(
        mesh_obj_idx, 
        init_transform=engine.primitives.objects[mesh_name].transform.model_matrix()
    )
    # Reset object coords in engine to frame 0
    deform_rendered_scene()
    # Sync geometry changes to 3dgrut engine
    engine_refresh()

# Update the global gaussians object given simulator transforms
def deform_rendered_scene():
    global engine  # just so we are clear what's mutated

    with torch.no_grad():
        # Update mesh positions
        mesh_deformed_pts = scene.get_object_deformed_pts(mesh_obj_idx, mesh_orig_pos).squeeze()
        engine.primitives.objects[mesh_name].vertices = mesh_deformed_pts
    
        # Update doll positions
        doll_transforms = scene.get_object_transforms(doll_obj_idx)
        # To support simulations with gaussians, 
        # we need to apply our affine transformations over the gaussian positions, rotation and scale.
        # The skinning weights were obtained by training the doll object.
        # The transforms are a 4x4 matrix, per skinning weight - a product of the simulation
        # The utility below takes care of applying these weighted transformations to Gaussians using Linear Blend Skinning
        new_pos, new_rot, new_scale = transform_gaussians_lbs(
            doll_gaussians.positions, doll_gaussians.rotation, doll_gaussians.scale,
            doll_skinning_weights, doll_transforms)
        engine.scene_mog.positions[doll_mask] = new_pos
        engine.scene_mog.rotation[doll_mask] = new_rot
        engine.scene_mog.scale[doll_mask] = new_scale

        # Debug: is table moving? It should not!
        # engine.scene_mog.positions[table_mask] = scene.get_object_deformed_pts(table_obj_idx, table_gaussians.get_positions()).squeeze()

    # Sync geometry changes to 3dgrut engine
    engine_refresh()

In [ ]:
scene.reset_scene()

#### Interactive Simulation Interface

Add a little GUI for the physics:

In [ ]:
num_sim_steps_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=300,
    step=1,
    orientation='horizontal',
    description='Steps',
    continuous_update=True,
    readout=True,
)

def run_sim_step():
    scene.run_sim_step()
    print(".", end="")
    with torch.no_grad():
        deform_rendered_scene()
        visualizer.render_update()

def run_simulation(b=None):
    if scene.current_sim_step == 0:

        # Uncomment lines below to try few different placements for the mesh and doll

        # Doll placements
        # scene.set_object_initial_transform(doll_obj_idx, init_transform=torch.tensor([[1,0,0,-0.3],[0,1,0,-0.2],[0,0,1,0.5]], dtype=torch.float32, device='cuda')) # doll falls on blue speakers
        # scene.set_object_initial_transform(doll_obj_idx, init_transform=torch.tensor([[1,0,0,0.0],[0,1,0,0.8],[0,0,1,0.5]], dtype=torch.float32, device='cuda'))   # doll falls in bag
        
        # Mesh placements
        # scene.set_object_initial_transform(mesh_obj_idx, init_transform=torch.tensor([[1,0,0,-1.8],[0,1,0,0.6],[0,0,1,1]], dtype=torch.float32, device='cuda')) # mesh on drill
        # scene.set_object_initial_transform(mesh_obj_idx, init_transform=torch.tensor([[1,0,0,-1.5],[0,1,0,1.5],[0,0,1,2]], dtype=torch.float32, device='cuda')) # mesh falls on boots
        # scene.set_object_initial_transform(mesh_obj_idx, init_transform=torch.tensor([[1,0,0,0.0],[0,1,0,-0.2],[0,0,1,1]], dtype=torch.float32, device='cuda')) # mesh falls on pliers
        # Use sliders to control mesh position
        scene.set_object_initial_transform(
            mesh_obj_idx, 
            init_transform=engine.primitives.objects[mesh_name].transform.model_matrix()
        )
        # Scene already takes care of transformation for us, so we can keep the transform at identity.
        # This will stop the engine from applying the transform twice..
        engine.primitives.objects[mesh_name].transform.reset()
        reset_transform_widgets_to_engine_values()
    num_simulation_steps = num_sim_steps_slider.value
    for s in range(num_simulation_steps):
        with visualizer.out:
            run_sim_step()

def reset_and_rerender(b=None):
    reset_simulation()
    visualizer.render_update()

# Create new tab for simulation
run_button = widgets.Button(description='Run Simulation')
run_button.on_click(run_simulation)
reset_button = widgets.Button(description='Reset Simulation')
reset_button.on_click(reset_and_rerender)
sim_buttons = widgets.HBox([run_button, reset_button])
simulation_controls = widgets.VBox([num_sim_steps_slider, sim_buttons])

# Recreate GUI component with an additional tab for simulation
tab_simulation = widgets.Box([simulation_controls])
gui_tab.children = [tab_renderer, tab_objects, tab_materials, tab_simulation]
gui_tab.set_title(0, 'Render Settings')
gui_tab.set_title(1, 'Objects')
gui_tab.set_title(2, 'PBR Materials')
gui_tab.set_title(3, 'Simulation')

Show the updated visualizer -- let's try running the simulation!

In [ ]:
reset_and_rerender() # Always start with initial conditions
visualizer_box = widgets.VBox([widgets.HBox([visualizer.canvas, gui_tab]), visualizer.out])
display(visualizer_box)

------

#### DEBUG SIMULATED POINT POSITIONS

The cell below visualizes the simulation from cubature point of view. This is useful for i.e. debugging collisions.

Feel free to uncomment and explore more about the behavior of the simulation!

In [ ]:
# import k3d
# import warp as wp
# plot = k3d.plot()

# def _get_sim_pts():
#     with torch.no_grad():
#         sim_rest_pts = wp.to_torch(scene.sim_pts)
#         log_tensor(sim_rest_pts, 'sim_rest_pts', print_stats=True)

#     dx = wp.array((scene.sim_B@scene.sim_z), dtype=wp.vec3)
#     dx_torch = wp.to_torch(dx)
#     log_tensor(dx_torch, 'dx_torch', print_stats=True)

#     return (sim_rest_pts + dx_torch).detach()

# sim_pts = _get_sim_pts()
# def _obj_idx(oid):
#     return wp.to_torch(scene.object_to_qp_map[oid]).detach()
    
# plot += k3d.points(sim_pts[_obj_idx(mesh_obj_idx)].detach().cpu().numpy(), point_size=collision_particle_radius, color=0xFF0000)
# plot += k3d.points(sim_pts[_obj_idx(doll_obj_idx)].detach().cpu().numpy(), point_size=collision_particle_radius, color=0x7700FF)
# plot += k3d.points(sim_pts[_obj_idx(table_obj_idx)].detach().cpu().numpy(), point_size=collision_particle_radius, color=0xCCCCCC)
# plot.display()

# Export High Quality Video

ℹ️ The last section shows how to use **Kaolin** cameras to create a trajectory and export a high quality video for offline viewing.

First, add buttons to save cameras from the visualizer.<br>Every time this button is pressed, the current visualizer camera is added to the list.

In [ ]:
import copy
cameras_to_record = []

# Add a button to the gui that populates this list
def add_cam(b):
    cam = copy.deepcopy(visualizer.camera)
    cameras_to_record.append(cam.cpu())
def reset_cams_button(b):
    cameras_to_record = []


add_cam_button = widgets.Button(description='📷 Add Camera')
add_cam_button.on_click(add_cam)
reset_cams_button = widgets.Button(description='Reset Trajectory')
reset_cams_button.on_click(add_cam)
export_vid_controls = widgets.HBox([add_cam_button, reset_cams_button])
tab_export_vid = widgets.Box([export_vid_controls])

# Recreate GUI component with an additional tab for exporting a video
gui_tab.children = [tab_renderer, tab_objects, tab_materials, tab_simulation, tab_export_vid]
gui_tab.set_title(0, 'Render Settings')
gui_tab.set_title(1, 'Objects')
gui_tab.set_title(2, 'PBR Materials')
gui_tab.set_title(3, 'Simulation')
gui_tab.set_title(4, 'Export Video')

In [ ]:
reset_and_rerender() # Always start with initial conditions
visualizer_box = widgets.VBox([widgets.HBox([visualizer.canvas, gui_tab]), visualizer.out])
display(visualizer_box)

Then, use these keyframe cameras to generate a full path.<br>
Once you're happy with the camera trajectory set above, run the section below to export the video.

To export the video, we'll traverse the interpolated path: we run the simulation and render frames for the video, in alternating steps.

In [ ]:
import math
import cv2
from tqdm import tqdm
from threedgrut_playground.utils.kaolin_future.interpolated_cameras import camera_path_generator

@torch.no_grad()
def run_offline_sim_step():
    scene.run_sim_step()
    deform_rendered_scene()

def render_smooth_trajectory(trajectory, output_path, frames_between_cameras, video_fps=30):
    if len(trajectory) < 2:
        raise ValueError('Rendering a path trajectory requires at least 2 cameras.')

    # Let kaolin generate an interpolated trajectory from the key cameras
    interpolated_path = camera_path_generator(
        trajectory=trajectory,
        frames_between_cameras=frames_between_cameras,
        interpolation='polynomial'
    )

    out_video = None
    reset_simulation()

    # Advance on trajectory and run simulation step as we go
    for idx, camera in enumerate(tqdm(interpolated_path)):
        # Render a frame
        rgb = engine.render(camera)['rgb']
        
        # Advance physics simulation
        if idx % 2 == 0:
            with torch.no_grad():
                scene.run_sim_step()
                deform_rendered_scene()

        # Save rendered frame to video
        if out_video is None:
            out_video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'),
                                        video_fps, (rgb.shape[2], rgb.shape[1]), True)
        data = rgb[0].clip(0, 1).detach().cpu().numpy()
        data = (data * 255).astype(np.uint8)
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
        out_video.write(data)
    out_video.release()

    print(f'Video exported to {output_path}')

if len(cameras_to_record) == 0:
    print('Add some cameras to export a high definition video!')
else:
    set_high_quality_mode()
    num_simulation_steps = num_sim_steps_slider.value
    num_frames_between_keyframes = 2 * (math.ceil(num_simulation_steps // len(cameras_to_record)) + 1)
    render_smooth_trajectory(cameras_to_record, output_path='output.mp4',frames_between_cameras=num_frames_between_keyframes, video_fps=30)

That's it! Your exported video should be ready!

Final words on cameras: Kaolin cameras are flexible. Throughout the tutorial we've used them to control the visualizer and interpolate keyframes for a video, but there is even more you can do with them!

See [the short examples page](https://github.com/NVIDIAGameWorks/kaolin/tree/master/examples/recipes/camera) on Github.

In [ ]:
kaolin.render.camera.Camera?